## INFERENCE TIME

In [18]:

import torch
import torchvision
from transformers import ViTImageProcessor
from torch.utils.data import DataLoader
from PIL import Image
import requests
import torch
import pickle
from transformers import ViTImageProcessor
from model import ViTLightningModule
from torchvision.transforms import (CenterCrop, 
                                    Compose, 
                                    Normalize, 
                                    RandomHorizontalFlip,
                                    RandomResizedCrop, 
                                    Resize, 
                                    ToTensor)


processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
image_mean = processor.image_mean
image_std = processor.image_std
size = processor.size["height"]

normalize = Normalize(mean=image_mean, std=image_std)


with open('id2label.pickle', 'rb') as handle:
    id2label = pickle.load(handle)

with open('label2id.pickle', 'rb') as handle:
    label2id = pickle.load(handle)

_val_transforms = Compose(
    [
        Resize(size),
        CenterCrop(size),
        ToTensor(),
        normalize,
    ]
)


model = ViTLightningModule(len(id2label), id2label, label2id)
model.load_state_dict(torch.load("./model"))
model.eval()



In [17]:
from PIL import Image
import requests
import torch

def predictExample(path: str, groundtruth: int):
    image = torchvision.datasets.folder.pil_loader(path)
    image_transformed = _val_transforms(image)

    with torch.no_grad():
      outputs = model(torch.unsqueeze(image_transformed,0))

    prediction = outputs.argmax(-1)
    print(f"Predicted {prediction} Real {groundtruth}")
    #print("Predicted class:", model.config.id2label[prediction.item()])


predictExample("Dataset/Colored/pos/HighresScreenshot00000.png", 1)
predictExample("Dataset/Colored/neg/HighresScreenshot00001.png", 0)


Predicted tensor([1]) Real 1
Predicted tensor([0]) Real 0
